In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load dos dados do ficheiro JSON e preparação dos dados para treino e teste
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    captured_data = data['capturedData']

    accelerometerXAxis = [entry['accelerometerXAxis'] for entry in captured_data]
    accelerometerYAxis = [entry['accelerometerYAxis'] for entry in captured_data]
    accelerometerZAxis = [entry['accelerometerZAxis'] for entry in captured_data]
    gyroscopeXAxis = [entry['gyroscopeXAxis'] for entry in captured_data]
    gyroscopeYAxis = [entry['gyroscopeYAxis'] for entry in captured_data]
    gyroscopeZAxis = [entry['gyroscopeZAxis'] for entry in captured_data]

    # criar dataFrame 
    df = pd.DataFrame({
        'accelerometerXAxis': accelerometerXAxis,
        'accelerometerYAxis': accelerometerYAxis,
        'accelerometerZAxis': accelerometerZAxis,
        'gyroscopeXAxis': gyroscopeXAxis,
        'gyroscopeYAxis': gyroscopeYAxis,
        'gyroscopeZAxis': gyroscopeZAxis
    })
    
    print("Dataframe:")
    print(df)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df.values)
    print("Scaled data:")
    print(scaled_data)

    sequence_length = 11
    sequences = []
    for i in range(len(scaled_data) - sequence_length):
        print(i)
        print(scaled_data[i:i+sequence_length])
        print("----")
        sequences.append(scaled_data[i:i+sequence_length])

    sequences = np.array(sequences)

    # conjuntos de treino e teste
    X = sequences[:, :-1]
    y = sequences[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, scaler

#treinar o modelo LSTM
def build_and_train_model(X_train, X_test, y_train, y_test):
    # construção do LSTM
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(X_train.shape[2])) 
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

    return model

# prever os valores e mostrar a comparação
def predict_and_compare(model, X_test, y_test, scaler):
    y_pred = model.predict(X_test)

    y_pred_inverse = scaler.inverse_transform(y_pred)
    y_test_inverse = scaler.inverse_transform(y_test)

    comparison_df = pd.DataFrame({'Real': y_test_inverse.flatten(), 'Predicted': y_pred_inverse.flatten()})

    difference = np.abs(comparison_df['Real'] - comparison_df['Predicted'])
    comparison_df['Absolute Difference'] = difference

    return comparison_df

def classify_acceleration(data):
    max_acceleration_0_10= 0.07
    
    df = pd.DataFrame(data['capturedData'])
    
    df['createdAt'] = pd.to_datetime(df['createdAt'])

    # converte para float
    df['speed Km/h'] = df['speed Km/h'].astype(float)
    
    # calcula a aceleracao com base na mudança de velocidade
    df['acceleration'] = df['speed Km/h'].diff() / df['createdAt'].diff().dt.total_seconds()

    # critérios para classificação 
    slow_threshold = 0.06 * max_acceleration_0_10
    aggressive_threshold = 0.08 * max_acceleration_0_10

    # funcao para classificar a aceleração
    def classify(acceleration):
        if acceleration >= aggressive_threshold:
            return 'aggressive'
        elif acceleration >= slow_threshold:
            return 'normal'
        else:
            return 'slow'

    # aplicar a função de classificação
    df['classification'] = df['acceleration'].apply(classify)

    return df[['acceleration', 'classification']]

def main():
    datasets = [
        # BMW e36 datasets 10m
        r'..\..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json',
        r'..\..\Datasets\Acceleration\Till reached distance\10m\normal\BMW e36\acceleration 0m 10m normal-23-04-2023-19-30-17.json',
        r'..\..\Datasets\Acceleration\Till reached distance\10m\slow\BMW e36\acceleration 0m 10m slow-23-04-2023-19-19-20.json',

    ]

    #comparação dos resultados
    models = []
    scalers = []
    comparison_dfs = []

    for dataset in datasets:
        print("Load e treino do modelo para:", dataset)
        X_train, X_test, y_train, y_test, scaler = load_dataset(dataset)
        model = build_and_train_model(X_train, X_test, y_train, y_test)
        models.append(model)
        scalers.append(scaler)

        # Carregamento dos dados específicos para classificação de velocidade
        with open(dataset, 'r') as file:
            data = json.load(file)
        classification_result = classify_acceleration(data)  # Classificar velocidade para os dados atuais
        print(classification_result)

        for i, model in enumerate(models):
            print("\n\nResultados para o Dataset:", datasets[i])
            X_train, X_test, y_train, y_test, scaler = load_dataset(datasets[i])
            comparison_df = predict_and_compare(model, X_test, y_test, scalers[i])
            comparison_dfs.append(comparison_df)
            print(comparison_df)  
            print("\n-----------------------------\n")  

if __name__ == "__main__":
    main()


Load e treino do modelo para: ..\..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json
Dataframe:
    accelerometerXAxis  accelerometerYAxis  accelerometerZAxis  \
0             0.204224            0.574472            0.015077   
1             0.230342            0.175526            0.122981   
2             0.199742            1.303585           -0.188207   
3             0.003083            0.753614            0.103433   
4             0.202767            0.281722            0.290354   
5            -0.042388           -0.066736           -0.030522   
6            -0.010837           -1.048308            0.204451   
7            -0.272801            1.421072           -0.017023   
8            -0.028238           -0.586086           -0.394835   
9            -0.077006            0.570721            0.579926   
10           -0.093719           -0.036977           -0.587654   
11           -0.043525            0.16

c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.2301 - val_loss: 0.2652
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.2035 - val_loss: 0.2385
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.1787 - val_loss: 0.2145
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.1557 - val_loss: 0.1933
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.1344 - val_loss: 0.1747
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.1147 - val_loss: 0.1586
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0968 - val_loss: 0.1450
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0806 - val_loss: 0.1338
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0663 - val_loss: 0.1251
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.0538 - val_loss: 0.1188
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0431 - val_loss: 0.1148
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0340 - val_loss: 0.1131
Ep

NameError: name 'classify_speed' is not defined